In [1]:
import cv2
import glob
import os
import numpy as np
from matplotlib import pyplot as plt
import albumentations as A

In [3]:
files = glob.glob("./ept_bugs/*/*.jpg")
#out_dir = "./ept_bugs_preproc/"

In [4]:
dirs = glob.glob("./ept_bugs/*")

for d in dirs:
    d_new = d.replace("ept_bugs", "ept_bugs_preproc")
    
    if not os.path.isdir(d_new):
        os.mkdir(d_new)

In [6]:
transforms = A.Compose([
    A.LongestMaxSize(max_size=224, always_apply=True, interpolation = cv2.INTER_LINEAR),
    A.PadIfNeeded(min_height=224, min_width=224, border_mode=cv2.BORDER_CONSTANT, always_apply=True)
])

In [13]:
for i in files:
    
    file_out = i.replace("ept_bugs", "ept_bugs_preproc")
    
    if os.path.isfile(file_out):
        continue
    
    img = cv2.imread(i)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    
    median = cv2.medianBlur(img, 5)
    hsv = cv2.cvtColor(median, cv2.COLOR_RGB2HSV)
    mask = cv2.inRange(hsv, (100, 0, 0), (140, 255,255))
    kernel = np.ones((5,5), np.uint8)
    mask_inv = 255 - mask
    mask_dilate = cv2.dilate(mask_inv, kernel, iterations=5)
    num_labels, labels, stats, centroids = cv2.connectedComponentsWithStats(mask_dilate, connectivity=8)
    stats_no_bg = stats[1:,]
    sizes = stats_no_bg[:,-1]
    
    if sizes.size == 0:
        print("Image "+ i +" is 0")
        continue
    
    bbox = stats_no_bg[np.argmax(sizes)]
    
    dim = mask_dilate.shape
    pad_constant = 100
    
    top = bbox[1] - pad_constant
    bottom = bbox[1] + bbox[3] + pad_constant
    left = bbox[0] - pad_constant
    right = bbox[0] + bbox[2] + pad_constant
    
    width = right-left
    height = bottom-top
    dim_diff = abs(height - width)
    pad = dim_diff//2
    
    uneven_corr = 0 if (dim_diff % 2) == 0 else 1

    if height > width:
        right = right+pad+uneven_corr
        left = left-pad
        
    if height < width:
        top = top-pad
        bottom = bottom+pad+uneven_corr
        
    if top < 0:
        top = 0
        
    if bottom > dim[0]:
        bottom = dim[0]
        
    if right > dim[1]:
        right = dim[0]
        
    if left < 0:
        left = 0
    
    img_crop = img[top:bottom, left:right, :]
    
    if img_crop.size == 0:
        print("Image "+ i +" is 0")
        continue
    
    img_trans = transforms(image=img_crop)
        
    img_out = cv2.cvtColor(img_trans["image"], cv2.COLOR_RGB2BGR)
    cv2.imwrite(file_out, img_out)

Image ./ept_bugs/Epeor/Epeor_33-specimen-i009-s056-e350.jpg is 0
Image ./ept_bugs/Calib/Calib_2-specimen-i005-s056-e250.jpg is 0
Image ./ept_bugs/Calib/Calib_26-specimen-i008-s056-e250.jpg is 0
Image ./ept_bugs/Culop/Culop_6-specimen-i001-s056-e150.jpg is 0
Image ./ept_bugs/Atops/Atops_70-specimen-i003-s056-e350.jpg is 0
Image ./ept_bugs/Asiop/Asiop_75-specimen-i004-s056-e150.jpg is 0
Image ./ept_bugs/Meg/Meg_29-specimen-i008-s056-e250.jpg is 0
Image ./ept_bugs/Fallc/Fallc_63-specimen-i001-s056-e150.jpg is 0
Image ./ept_bugs/Cerat/Cerat_78-specimen-i008-s056-e250.jpg is 0
Image ./ept_bugs/Cerat/Cerat_85-specimen-i008-s056-e250.jpg is 0
Image ./ept_bugs/Cerat/Cerat_94-specimen-i005-s056-e250.jpg is 0
Image ./ept_bugs/Cerat/Cerat_94-specimen-i002-s056-e250.jpg is 0
Image ./ept_bugs/Cerat/Cerat_90-specimen-i003-s056-e350.jpg is 0
Image ./ept_bugs/Baets/Baets_33-specimen-i008-s056-e250.jpg is 0
Image ./ept_bugs/Baets/Baets_1-specimen-i001-s056-e150.jpg is 0
